## Set up the pipeline resources

The Azure ML framework can be used from CLI, Python SDK, or studio interface. In this example, you'll use the AzureML Python SDK v2 to create a pipeline. 

Before creating the pipeline, you'll set up the resources the pipeline will use:

* The dataset for training
* The software environment to run the pipeline

## Connect to the workspace

Before we dive in the code, you'll need to connect to your Azure ML workspace. The workspace is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning.

We are using `DefaultAzureCredential` to get access to workspace. 
`DefaultAzureCredential` should be capable of handling most Azure SDK authentication scenarios. 

Reference for more available credentials if it does not work for you: [configure credential example](../../configuration.ipynb), [azure-identity reference doc](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python).

In [ ]:
import sys

print(sys.prefix)

In [ ]:
# Handle to the workspace
from azure.ai.ml import MLClient

# Authentication package
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

In the next cell, enter your Subscription ID, Resource Group name and Workspace name. To find your Subscription ID:
1. In the upper right Azure Machine Learning Studio toolbar, select your workspace name.
1. At the bottom, select **View all properties in Azure Portal**
1. Copy the value from Azure Portal into the code.

In [ ]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

The result is a handler to the workspace that you'll use to manage other resources and jobs.

> [!IMPORTANT]
> Creating MLClient will not connect to the workspace. The client initialization is lazy, it will wait for the first time it needs to make a call (in the notebook below, that will happen during dataset registration).


## Deploy the model as batch endpoint

Now deploy your machine learning model as a web service in the Azure cloud, an [`batch endpoint`](https://docs.microsoft.com/azure/machine-learning/concept-endpoints).

To deploy a machine learning service, you usually need:

* The entry script must understand the data that the model expects and returns. When using a MLFlow model, as in this tutorial, this script is automatically created for you. Samples of scoring scripts can be found [here](https://github.com/Azure/azureml-examples/tree/sdk-preview/sdk/endpoints/online).



## Create a new batch endpoint

Now that you have a registered model and an inference script, it's time to create your online endpoint. The endpoint name needs to be unique in the entire Azure region. For this tutorial, you'll create a unique name using [`UUID`](https://en.wikipedia.org/wiki/Universally_unique_identifier#:~:text=A%20universally%20unique%20identifier%20(UUID,%2C%20for%20practical%20purposes%2C%20unique.).

In [ ]:
import datetime

endpoint_name = "batch-" + datetime.datetime.now().strftime("%m%d%H%M%f")
endpoint_name

In [ ]:
from azure.ai.ml.entities import BatchEndpoint

# create a batch endpoint
endpoint = BatchEndpoint(
    name=endpoint_name,
    description="A batch endpoint for predicting mobile prices",
)

ml_client.batch_endpoints.begin_create_or_update(endpoint)

In [ ]:
import os
import sys

project_root_directory = os.getcwd().split("/notebooks")[0]
sys.path.insert(0, project_root_directory)

In [ ]:
from core.modeling.deployment import DeploymentPipeline
from core.modeling.config import DeploymentConfig

In [ ]:
model = DeploymentPipeline(DeploymentConfig).load_registered_model(
    model_name="mobile-price"
)

for more details about the configuration

https://learn.microsoft.com/en-us/python/api/azure-ai-ml/azure.ai.ml.entities.batchdeployment?view=azure-python

In [ ]:
from azure.ai.ml.entities import BatchDeployment, BatchRetrySettings
from azure.ai.ml.constants import BatchDeploymentOutputAction

deployment = BatchDeployment(
    name=name
    description="regressor-mobile-mlflow",
    endpoint_name=endpoint.name,
    model=model.id,
    compute="sandbox-ci",
    settings=ModelBatchDeploymentSettings(
        max_concurrency_per_instance=2,
        mini_batch_size=10,
        instance_count=2,
        output_action=BatchDeploymentOutputAction.APPEND_ROW,
        output_file_name="predictions.csv",
        retry_settings=BatchRetrySettings(max_retries=3, timeout=30),
        logging_level="info",
    ),
)


In [ ]:
ml_client.batch_deployments.begin_create_or_update(deployment)

Now that you have deployed a model to a batch endpoint, and have an unlabeled data asset, you're ready to invoke the endpoint to generate predictions on the unlabeled data.

First, you'll define the input by referring to the registered data asset. Then, you'll invoke the endpoint, which will submit a pipeline job. You can use the job URL to monitor it in the Studio. The job will contain a child job that represents the running of the (generated) scoring script to get the predictions.

In [ ]:
deployment.name

In [ ]:
endpoint.defaults = {}

endpoint.defaults["deployment_name"] = deployment.name

ml_client.batch_endpoints.begin_create_or_update(endpoint)

In [ ]:
endpoint.defaults = {}

endpoint.defaults["deployment_name"] = deployment.name

ml_client.batch_endpoints.begin_create_or_update(endpoint)

### Test with a sample query

Now that the model is deployed to the endpoint, you can run inference with it.

Create a sample request file following the design expected in the run method in the score script.

In [ ]:
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.entities import Data

data_path = "../data/sample.csv"
dataset_name = "mobile-price-sample-endpoint"

mobile_price_sample = Data(
    path=data_path,
    type=AssetTypes.URI_FILE,
    description="Sample to test the endpoint.",
    name=dataset_name,
)

In [ ]:
ml_client.data.create_or_update(mobile_price_sample)

In [ ]:
dataset_name

In [ ]:
mobile_price_sample = ml_client.data.get(name=dataset_name, label="latest")

### Get logs of the deployment

Check the logs to see whether the endpoint/deployment were invoked successfully. If you face errors, see Troubleshooting online endpoints deployment.

In [ ]:
from azure.ai.ml import Input

input = Input(path=mobile_price_sample.id)

In [ ]:
# test the blue deployment with some sample data
job = ml_client.batch_endpoints.invoke(
    endpoint_name=endpoint_name,
    input=input,
    deployment_name=deployment.name,
)

In [ ]:
ml_client.jobs.get(job.name)

In [ ]:
logs = ml_client.online_deployments.get_logs(
    name="blue", endpoint_name=online_endpoint_name, lines=50
)
print(logs)

## Clean up resources

If you're not going to use the endpoint, delete it to stop using the resource.  Make sure no other deployments are using an endpoint before you delete it.


> [!NOTE]
> Expect this step to take approximately 6 to 8 minutes.

In [ ]:
ml_client.batch_endpoints.begin_delete(name=endpoint_name)